.. _API Tutorial:

# API Tutorial

This is a walkthrough of using the ``mutatest`` API.
These are the same method calls used by the CLI and provide additional flexibility for customization.
The code and notebook to generate this tutorial is located under the ``docs/api_tutorial`` folder on GitHub.

In [1]:
import ast

from pathlib import Path

from mutatest import run
from mutatest import transformers
from mutatest.api import Genome, GenomeGroup
from mutatest.filters import CoverageFilter, CategoryCodeFilter

## Setup

The ``example/`` folder has two Python files, ``a.py`` and ``b.py``, with a ``test_ab.py`` file that would be automatically detected by ``pytest``.

In [2]:
src_loc = Path("example")

In [3]:
print(*src_loc.iterdir(), sep="\n")

example/a.py
example/__pycache__
example/test_ab.py
example/b.py


## Run a clean trial and generate coverage

We can use ``run`` to perform a "clean trial" of our test commands based on the source location. 
This will generate a ``.coverage`` file that will be used by the ``Genome``. 
A ``.coverage`` file is not required.

In [4]:
# The return value of clean_trial is the time to run
# this is used in reporting from the CLI

run.clean_trial(src_loc, test_cmds=["pytest", "--cov=example"])

datetime.timedelta(microseconds=440740)

In [5]:
Path(".coverage").exists()

True

## Genome Basics

``Genomes`` are the basic representation of a source code file in ``mutatest``.
They can be initialized by passing in the path to a specific file, or initialized without any arguments and have the source file added later. 
The basic properties include the Abstract Syntax Tree (AST), the source file, the coverage file, and any category codes for filtering.

In [6]:
# Initialize with the source file location
# By default, the ".coverage" file is set for the coverage_file property
genome = Genome(src_loc / "a.py")

In [7]:
genome.source_file

PosixPath('example/a.py')

In [8]:
genome.coverage_file

PosixPath('.coverage')

In [9]:
# By default, no filter codes are set
# These are categories of mutations to filter
genome.filter_codes

set()

### Finding mutation targets

The ``Genome`` has two additional properties related to finding mutation locations: ``targets`` and ``covered_targets``.
These are sets of ``LocIndex`` objects (defined in ``transformers``) that represent locations in the AST
that can be mutated. Covered targets are those that have lines covered by the set ``coverage_file`` property. 

In [10]:
genome.targets

{LocIndex(ast_class='BinOp', lineno=5, col_offset=11, op_type=<class '_ast.Add'>),
 LocIndex(ast_class='Compare', lineno=8, col_offset=11, op_type=<class '_ast.Gt'>)}

In [11]:
genome.covered_targets

{LocIndex(ast_class='BinOp', lineno=5, col_offset=11, op_type=<class '_ast.Add'>)}

In [12]:
genome.targets - genome.covered_targets

{LocIndex(ast_class='Compare', lineno=8, col_offset=11, op_type=<class '_ast.Gt'>)}

### Accessing the AST

The ``ast`` property is the AST of the source file.
You can access the properties directly. 
This is used to generate the targets and covered targets through ``transformers.MutateAST``.

In [13]:
genome.ast

In [14]:
genome.ast.body

In [15]:
genome.ast.body[1].__dict__

{'name': 'add_five',
 'args': <_ast.arguments at 0x7f8190310438>,
 'body': [<_ast.Return at 0x7f81903104a8>],
 'decorator_list': [],
 'returns': None,
 'lineno': 4,
 'col_offset': 0}

### Filtering mutation targets

You can set filters on a ``Genome`` for specific types of targets. 
For example, setting ``bn`` for ``BinOp`` will filter both targets and covered targets to only ``BinOp`` class operations.

In [16]:
# All available categories are listed in transformers.CATEGORIES
print(*[f"Category:{k}, Code: {v}"
        for k,v in transformers.CATEGORIES.items()],
      sep="\n")

Category:AugAssign, Code: aa
Category:BinOp, Code: bn
Category:BinOpBC, Code: bc
Category:BinOpBS, Code: bs
Category:BoolOp, Code: bl
Category:Compare, Code: cp
Category:CompareIn, Code: cn
Category:CompareIs, Code: cs
Category:If, Code: if
Category:Index, Code: ix
Category:NameConstant, Code: nc
Category:SliceUS, Code: su
Category:SliceRC, Code: sr


In [17]:
# If you attempt to set an invalid code a ValueError is raised
# and the valid codes are listed in the message

try:
    genome.filter_codes = ("asdf",)
    
except ValueError as e:
    print(e)

Invalid category codes: {'asdf'}.
Valid codes: {'AugAssign': 'aa', 'BinOp': 'bn', 'BinOpBC': 'bc', 'BinOpBS': 'bs', 'BoolOp': 'bl', 'Compare': 'cp', 'CompareIn': 'cn', 'CompareIs': 'cs', 'If': 'if', 'Index': 'ix', 'NameConstant': 'nc', 'SliceUS': 'su', 'SliceRC': 'sr'}


In [18]:
# Set the filter using an iterable of the two-letter codes

genome.filter_codes = ("bn",)

In [19]:
# Targets and covered targets will only show the filtered value

genome.targets

{LocIndex(ast_class='BinOp', lineno=5, col_offset=11, op_type=<class '_ast.Add'>)}

In [20]:
genome.covered_targets

{LocIndex(ast_class='BinOp', lineno=5, col_offset=11, op_type=<class '_ast.Add'>)}

In [21]:
# Reset the filter_codes to an empty set
genome.filter_codes = set()

In [22]:
# All target classes are now listed again

genome.targets

{LocIndex(ast_class='BinOp', lineno=5, col_offset=11, op_type=<class '_ast.Add'>),
 LocIndex(ast_class='Compare', lineno=8, col_offset=11, op_type=<class '_ast.Gt'>)}

### Changing the source file in a Genome

If you change the source file property of the ``Genome`` all core properties except the coverage file and filters are reset - this includes targets, covered targets, and AST.

In [23]:
genome.source_file = src_loc / "b.py"

In [24]:
genome.targets

{LocIndex(ast_class='CompareIs', lineno=5, col_offset=11, op_type=<class '_ast.Is'>)}

In [25]:
genome.covered_targets

{LocIndex(ast_class='BinOp', lineno=5, col_offset=11, op_type=<class '_ast.Add'>)}